In [1]:
import os
import torch
# Set GPU device
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

os.environ['http_proxy']  = 'http://192.41.170.23:3128'
os.environ['https_proxy'] = 'http://192.41.170.23:3128'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [6]:
# Loaders
from langchain.schema.document import Document
from langchain_community.document_loaders import PyMuPDFLoader
# Model
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
# Embedding Support
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores import Chroma
# Agent & Tool
from langchain.agents.agent_toolkits.vectorstore.base import create_vectorstore_agent
from langchain.agents.agent_toolkits.vectorstore.toolkit import VectorStoreToolkit, VectorStoreInfo
from langchain.agents.agent_toolkits.vectorstore.prompt import PREFIX, ROUTER_PREFIX

from dotenv import load_dotenv
load_dotenv()

from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_organization = os.getenv("OPENAI_API_ORG")

In [7]:
llm = ChatOpenAI(
    model_name  = "gpt-3.5-turbo-16k",
    temperature = 0, 
    max_tokens  = 4096,
    openai_api_key=openai_api_key, 
    openai_organization=openai_organization
)
embedding_model = OpenAIEmbeddings(
    openai_api_key=openai_api_key, 
    openai_organization=openai_organization
)

In [8]:
pdf_path = './docs/LLaVA.pdf' 
pdf_loader = PyMuPDFLoader(pdf_path)
pages = pdf_loader.load_and_split()

vectorstore = FAISS.from_documents(
    pages, 
    embedding=embedding_model)

In [10]:
# Create vectorstore info object
vectorstore_info = VectorStoreInfo(
    name        = 'vectorStoreTool',
    description = 'a sample vectorstore',
    vectorstore = vectorstore,)

# Convert vectorstore info object to agent toolkit
toolkit = VectorStoreToolkit(
    vectorstore_info = vectorstore_info, 
    llm = llm, )

In [11]:
# Create the agent executor
PREFIX = """You are an agent designed to answer questions about sets of documents.
You have access to tools for interacting with the documents, and the inputs to the tools are questions.
"""
user_prompt = 'Summary all documents'
agent_executor = create_vectorstore_agent(
    llm=llm, toolkit=toolkit, prefix=PREFIX, verbose=True)

result = agent_executor.run(user_prompt)


/opt/homebrew/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAI instead.
  warn_deprecated(
/opt/homebrew/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAI instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
To summarize all documents, I should use the vectorStoreTool.
Action: vectorStoreTool
Action Input: "Summary all documents"
Observation: The documents provided contain information about evaluating the performance of GPT-4 (a language model) on image-related tasks. Table 3 shows the relative scores of GPT-4 compared to other settings on different types of questions. The instructions for brief and detailed image descriptions are listed in Tables 8 and 9 respectively. The CC3M dataset is described, which involves extracting noun-phrases from captions. A travel blog post prompt is given, asking for an engaging post about a trip. Figure 3 demonstrates LLaVA's ability to recognize visual content and provide detailed responses. The limitations of a simplified version of GPT-4 are discussed, and the use of language-only GPT-4 or ChatGPT to create instruction-following data involving visual content is explained.
Thought:I now know the summary of all the d

In [16]:
result

"The documents contain information about evaluating the performance of GPT-4 on image-related tasks, instructions for image descriptions, details about the CC3M dataset, a travel blog post prompt, demonstrations of LLaVA's ability to recognize visual content, and discussions about the limitations of a simplified version of GPT-4 and creating instruction-following data involving visual content."